In [4]:
import tensorflow as tf
import numpy as np


In [5]:
latent_dim = 32
height = 32
width = 32
channels = 3


In [6]:
# network for generator

# creating inputs from noise

generator_input = tf.keras.Input(shape=(latent_dim, ))

# converting inputs into a feature map of 16 * 16, 128c 

x = tf.keras.layers.Dense(128 * 16 * 16)(generator_input)
x = tf.keras.layers.LeakyReLU()(x)
x = tf.keras.layers.Reshape((16, 16, 128))(x)

# adding a convolutional layer

x = tf.keras.layers.Conv2D(256, 4, padding='same')(x)
x = tf.keras.layers.LeakyReLU()(x)

# up-sampling as 32 * 32

x = tf.keras.layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = tf.keras.layers.LeakyReLU()(x)

# adding convolutional layers

x = tf.keras.layers.Conv2D(256, 5, padding='same')(x)
x = tf.keras.layers.LeakyReLU()(x)
x = tf.keras.layers.Conv2D(256, 5, padding='same')(x)
x = tf.keras.layers.LeakyReLU()(x)

# creating a feature map of 32 * 32, 1c

x = tf.keras.layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)

# instantiate a generator model
# mapping an input of (latent_dim, ) shape into an image of (32, 32, 3)

generator = tf.keras.models.Model(generator_input, x)
generator.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32768)             0         
_________________________________________________________________
reshape (Reshape)            (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 16, 16, 256)       524544    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 32, 32, 256)       1048832   
__________

In [7]:
# network for discriminator

discriminator_input = tf.keras.layers.Input(shape=(height, width, channels))

x = tf.keras.layers.Conv2D(128, 3)(discriminator_input)
x = tf.keras.layers.LeakyReLU()(x)
x = tf.keras.layers.Conv2D(128, 4, strides=2)(x)
x = tf.keras.layers.LeakyReLU()(x)
x = tf.keras.layers.Conv2D(128, 4, strides=2)(x)
x = tf.keras.layers.LeakyReLU()(x)
x = tf.keras.layers.Conv2D(128, 4, strides=2)(x)
x = tf.keras.layers.LeakyReLU()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(0.4)(x)  # dropout
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)  # layer for classification

# instantiate a discriminator model
# implement the binary classification for the inputs of (32, 32, 3) configuration

discriminator = tf.keras.models.Model(discriminator_input, x)
discriminator.summary()

# setting an optimizer
# using decay of learning rate to stabilize the learning

discriminator_optimizer = tf.keras.optimizers.RMSprop(lr=0.0008,
                                                   clipvalue=1.0,
                                                   decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer,
                      loss='binary_crossentropy')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 6, 6, 128)         0         
__________

In [8]:
# adversarial network

# disable training when combined with generator

discriminator.trainable = False

# set DCGAN
gan_input = tf.keras.Input(shape=(latent_dim,))  # noise
gan_output = discriminator(generator(gan_input))
gan = tf.keras.models.Model(gan_input, gan_output)

# compile DCGAN
gan_optimizer = tf.keras.optimizers.RMSprop(lr=0.0004,
                                         clipvalue=1.0,
                                         decay=1e-8)
gan.compile(optimizer=gan_optimizer,
            loss='binary_crossentropy')


In [26]:
# implementing the code for training of GAN

import os

# downloading CIFAR10
(x_train, y_train), (_, _) = tf.keras.datasets.cifar10.load_data()

# choosing class 6 (flog)
x_train = x_train[y_train.flatten() == 6]

# normalization of data
x_train = x_train.reshape(
    (x_train.shape[0], ) + (height, width, channels)).astype('float32') / 255.

iterations = 10000
batch_size = 20

# designating a folder for generated images
# save_dir = '/C:/Users/hiro/AnacondaProjects/research/'

start = 0
for step in range(iterations):  # starting learning loops 
    # extracting points at random from a latent space 
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    
    # decoding to fake images
    generated_images = generator.predict(random_latent_vectors)
    
    # mixing all data including reals and fakes
    stop = start + batch_size
    real_images = x_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images])
    
    # creating labels to distinguish reals and fakes / real = 0, fake = 1
    labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
    
    # adding some noise to labels
    labels += 0.05 * np.random.random(labels.shape)
    
    # training discriminator model
    d_loss = discriminator.train_on_batch(combined_images, labels)
    
    # extracting points at random form a latent space
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    
    # creating labels representing all reals
    misleading_targets = np.zeros((batch_size, 1))
    
    # training GAN model
    # (weight of discriminator is fixed)
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    
    start += batch_size
    if start > len(x_train) - batch_size:
        start = 0
    
    if step % 100 == 0:  ## saving and plotting at every 10 steps
        # gan.save_weights('DCGAN.h5')
    
        # outputting the performances
        print('discriminator loss at step {}: {}'.format(step, d_loss))
        print('adversarial loss at step {}: {}'.format(step, a_loss))
    
        # # saving one generated image
        # img = image.array_to_img(generated_images[0] * 255., scale=False)
        # img.save(os.path.join(save_dir, 'generated_frog' + str(step) + '.png'))
        # 
        # # saving one real image for comparison
        # img = tf.keras.preprocessing.image.array_to_img(real_images[0] * 255., scale=False)
        # img.save(os.path.join(save_dir, 'real_frog' + str(step) + '.png'))        


discriminator loss at step 0: 0.6914457082748413
adversarial loss at step 0: 0.6591678857803345


KeyboardInterrupt: 